In [ ]:
import torchao
from torchao.dtypes import NF4Tensor
from torchao.dtypes.nf4tensor import to_nf4

In [ ]:
import torch
x = torch.randn(16, 16)  # Example full precision tensor
x_nf4 = to_nf4(x, 16, 1)

In [ ]:
x - x_nf4.get_original_weight()

In [ ]:
# lets compile a basic example
@torch.compile(fullgraph = True)
def compiled_get_original(x):
  return x.get_original_weight()

In [ ]:
compiled_get_original(x_nf4)